In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install faster-whisper==1.0.3 flask pyngrok pydub soundfile
!apt-get -y install ffmpeg

  Using cached faster_whisper-1.0.3-py3-none-any.whl.metadata (15 kB)
  Using cached pyngrok-7.3.0-py3-none-any.whl.metadata (8.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 51.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 45.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 90.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [5]:
from pyngrok import ngrok
ngrok.set_auth_token("31JvDkD7grsF2LO5oiprZglqqLW_7meFpK9M2Em2RtaSo81o4")

In [ ]:
import os
import tempfile
from flask import Flask, request, jsonify, send_file
from pydub import AudioSegment
from faster_whisper import WhisperModel

# ==== Model load ====
MODEL_NAME = os.environ.get("WHISPER_MODEL", "large-v3")
COMPUTE_TYPE = os.environ.get("WHISPER_COMPUTE", "float16")
DEVICE = "cuda"
model = WhisperModel(MODEL_NAME, device=DEVICE, compute_type=COMPUTE_TYPE)

# ==== Preprocessing ====
def preprocess_to_wav_mono16k(in_path: str) -> str:
    """Convert any audio to mono 16k WAV for Whisper."""
    audio = AudioSegment.from_file(in_path)
    audio = audio.set_channels(1).set_frame_rate(16000)
    out = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    audio.export(out.name, format="wav")
    return out.name

# ==== Transcription ====
def transcribe_file_simple(in_path: str, language="en"):
    print("Starting transcription...")
    wav = preprocess_to_wav_mono16k(in_path)
    segments, info = model.transcribe(
        wav,
        language=language,
        vad_filter=True,
        beam_size=5,
        temperature=0
    )
    print("Transcription finished.")
    all_text = " ".join([s.text for s in segments]).strip()
    return {"text": all_text, "language": getattr(info, "language", language)}

# ==== Flask app ====
app = Flask(__name__)

@app.route("/transcribe", methods=["POST"])
@app.route("/transcribe", methods=["POST"])
def transcribe_endpoint():
    import os  # Ensure os is imported
    if "file" not in request.files:
        return jsonify({"error": "No file uploaded"}), 400

    file = request.files["file"]

    with tempfile.NamedTemporaryFile(suffix=os.path.splitext(file.filename)[1], delete=False) as tmp:
        file.save(tmp.name)
        print("File saved, starting transcription...")
        result = transcribe_file_simple(tmp.name)
        print("Transcription done.")

    # Save transcript into /kaggle/working
    out_path = "/kaggle/working/transcript.txt"
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(result.get("text", ""))
        f.flush()
        os.fsync(f.fileno())
        print("Transcript written to disk.")

    # Send JSON with transcript and link
    print("Returning response to client.")
    return jsonify({
        "status": "done",
        "download_url": "/download/transcript.txt"
    })
    if "file" not in request.files:
        return jsonify({"error": "No file uploaded"}), 400

    file = request.files["file"]

    with tempfile.NamedTemporaryFile(suffix=os.path.splitext(file.filename)[1], delete=False) as tmp:
        file.save(tmp.name)
        result = transcribe_file_simple(tmp.name)

    # Save transcript into /kaggle/working
    out_path = "/kaggle/working/transcript.txt"
    with open(out_path, "w", encoding="utf-8") as f:
    f.write(result.get("text", ""))
    f.flush()
    os.fsync(f.fileno())

    # Send JSON with transcript and link
    return jsonify({
        "status": "done",
        "download_url": "/download/transcript.txt"
    })

@app.route("/download/<path:filename>")
def download(filename):
    full_path = os.path.join("/kaggle/working", filename)
    if not os.path.exists(full_path):
        return jsonify({"error": "File not found"}), 404
    return send_file(full_path, as_attachment=True)

# ==== ngrok tunnel ====
public_url = ngrok.connect(5000, "http")  # use http to reduce SSL errors
print(f"Public URL: {public_url}")

if __name__ == "__main__":
    app.run(port=5000)
